# HW _ 3 : Emperical Analysis
# Turn in for: Rahul Deshpande (deshpr)

### Step 1. Ensure the required packages are installed.

In [74]:
# Display the directory from where we are getting the packages.
.libPaths()
# One or more of the commands below will fail if the library is not installed.
library(e1071)
library(caret)
library(class)
library(rpart)
#install.packages('e1071', dependencies=TRUE)
require('e1071')
#install.packages('caret', dependencies = TRUE)
require(caret)

[1] "C:/Users/radeshpa/AppData/Local/Programs/Python/Python37/Lib/site-packages/rpy2/R/win-library/3.5"
[2] "C:/Program Files/R/R-3.5.2/library"

### Step 2. Load the Gene Expression Data

In [75]:
# To measure time (for debugging)
start.time <- Sys.time()
geneExpressionFileName  <- 'gene_expression_n438x978.txt' 
myData <- read.table(geneExpressionFileName, sep="\t", header=T)
dimnames(myData)[[1]] <- myData[,1]
myData <- myData[,-1]
myData[1:5, 1:5]
end.time <- Sys.time()
cat("took time: ",end.time - start.time)
dim(myData)

,PSME1,ATF1,RHEB,FOXO3,RHOA
ACETAZOLAMIDE,-0.015159099,-0.031470528,-0.004733488,0.02591061,0.0056296773
IRBESARTAN,-0.026811981,0.012151979,-0.025550148,-0.02401181,-0.0106717396
IPRATROPIUM BROMIDE,0.001017958,-0.008650622,-0.018128698,-0.02079971,-0.0002722781
EFAVIRENZ,-0.004398264,0.055387992,0.004658520,0.01380732,-0.0340697348
THIAMINE,0.001838965,-0.018079188,-0.011855532,-0.03705033,-0.0133954959


took time:  0.779546

[1] 438 978

#### We see that we have 978 genes, and 438 examples. 

### Step 3. Load the Adverse Drug Side Effect Data

In [76]:
sideEffectFileName <- 'ADRs_HLGT_n438x232.txt'
sideEffectData <- read.table(sideEffectFileName, sep="\t", header=T)
dimnames(sideEffectData)[[1]] <- sideEffectData[,1]
sideEffectData <- sideEffectData[,-1]
sideEffectData[1:5, 1:5]
dim(sideEffectData)
names <- names(sideEffectData)

,Abdominal.hernias.and.other.abdominal.wall.conditions,Abortions.and.stillbirth,Acid.base.disorders,Administration.site.reactions,Adrenal.gland.disorders
ACETAZOLAMIDE,0,0,1,1,0
IRBESARTAN,0,0,0,0,1
IPRATROPIUM BROMIDE,0,0,0,0,1
EFAVIRENZ,0,0,0,0,1
THIAMINE,0,0,0,0,0


[1] 438 232

#### We see we have 232 side effects and 438 examples. So if we combine this with the gene data, it is like saying we have 438 examples,
####  and 232 different  outcome variables.

### Step 4. Some more data refining

#### Make sure there are no NAs in the data, otherwise imputing might be required.

In [77]:
anyNA(myData)
anyNA(sideEffectData)

[1] FALSE

[1] FALSE

#### We see there are no NAs in the data. Therefore, we can begin with performing our experiments.

# Step 5. Experiments

### We now begin performing experiments on our data. Each cell block below performs a different experiment, 
### and outputs the results to a .csv file - this helps with recording results  for the final .csv file.

### Experiment 5.1, t-test with p-value < 0.03, SVM Classification

In [ ]:
start.time <- Sys.time()
sideEffectNames <- names(sideEffectData)
geneNames <- names(myData)
experimentName <- "t_0_03_svm"
experimentResults <- data.frame(a = character(), b= numeric())
experimentResults <- rbind(experimentResults, data.frame(a = "side effect name", b = 0))

################################### EXPERIMENT OUTPUT FILE NAME ################################################################
fileName <- paste(experimentName, '.csv')
################################################################################################################################

experimentFile <- paste("C:\\Users\\radeshpa\\Desktop\\BioInformatics\\Assignment3\\Experiment_output", fileName)

for(i in 1:length(sideEffectNames)){
    sideEffectName <- sideEffectNames[i]
    cat("Working on the side effect: ", sideEffectName, "\n")
    currentSideEffect <- sideEffectData[,sideEffectName]
    presentIndices <- which(currentSideEffect == 1)
    absentIndices <- which(currentSideEffect == 0)
   
    outerCount <- 3
    accuracyOverFoldsTotal <- 0
    
    for(j in 1:outerCount){
        
        
################################################  DATA  #######################################################################
        
        foldCount <- 10
        folds <- createFolds(currentSideEffect, foldCount, returnTrain = FALSE)
        # Get the values for every list (indices)

        foldIndices <- list()
        
        for(k in 1:length(folds)){
            foldIndices[[length(foldIndices)+1]] <- folds[[k]]
        }

        accuracyOverFolds <- 0
        accuracyOverFoldsSum <- 0
       
        
        for(k in 1:length(folds)){
            # Data of this fold.
            cat("Running fold number: ", k, " Count number: ", j, "\n")

            # We need to create a vector for indexing
            testingDataX <- myData[foldIndices[[k]],]


            testingDataY <- sideEffectData[foldIndices[[k]],sideEffectName]


            remainingIndices <- (Reduce(c,foldIndices[-k]))

            trainingDataX <- myData[c(remainingIndices), ]

            trainingDataY <- sideEffectData[c(remainingIndices), sideEffectName]

##################################################### FEATURE SELECTION : t- statistic ###########################################################            

            allpValues <- sapply(geneNames, function(geneName){
                geneData <- trainingDataX[,geneName]
                sampleOne <-  geneData[presentIndices]
                sampleOneLength <- length(sampleOne)
                sampleTwo <-  geneData[absentIndices]
                sampleTwoLength <- length(sampleTwo)
                if(sampleOneLength == 0 | sampleTwoLength == 0 | (sampleOneLength == 1 && sampleTwoLength != 1) | (sampleOneLength != 1 && sampleTwoLength == 1)){
                    pValue  <- 0 # we want this feature.
                }
                else{
                    pValue <- t.test(sampleOne, sampleTwo)$p.value
                }
                pValue            
            })

            ######## THRESHOLD FOR THE STATISTIC #############
            
            threshold <- 0.03
            sigpIndices <- which(allpValues < threshold)

            sigpValues <- allpValues[sigpIndices]
            featuresOfInterest <- names(trainingDataX)[sigpIndices]

            refinedTrainingDataX <- trainingDataX[, featuresOfInterest]

            
########################################## SVM #######################################################################################

#            cat("Features of interest: ", length(featuresOfInterest), "\n")
            # Now filter the test set also!
            refinedTestingDataX <- testingDataX[,featuresOfInterest]
            svmModel <- svm(refinedTrainingDataX, trainingDataY)
            predictionResults <- predict(svmModel, refinedTestingDataX)
#            cat("Count of prediction results: ", length(predictionResults))

            probabilityThreshold <- 0.5
            predictionResults <- round(predictionResults) # default is 0.5
            accuracy <- (sum(predictionResults==testingDataY) + sum(is.na(predictionResults) & is.na(testingDataY))) / length(predictionResults)
            #cat("Accuracy for fold:  ", i, " is ", accuracy, "\n")
            accuracyOverFoldsSum <- accuracyOverFoldsSum + accuracy
     
#######################################################################################################################        

        }
        
        accuracyOverFolds <- (accuracyOverFoldsSum)/length(folds)     
        accuracyOverFoldsTotal <- accuracyOverFoldsTotal + accuracyOverFolds
    }
    
    finalAccuracy  <- accuracyOverFoldsTotal/outerCount
    cat("Final Accuracy is: ", finalAccuracy, "\n")
    
    write.table(data.frame(a = sideEffectName, b = finalAccuracy), experimentFile, append = TRUE, sep = "\t", row.names = FALSE, col.names = FALSE)    
}

end.time <- Sys.time()
cat("took time: ",end.time - start.time)

### Experiment 5.2, Correlation of 0.05, -0.05, SVM

In [ ]:
start.time <- Sys.time()
sideEffectNames <- names(sideEffectData)
geneNames <- names(myData)
experimentName <- "correlation_0_05_SVM_Accuracy"
experimentResults <- data.frame(a = character(), b= numeric())
experimentResults <- rbind(experimentResults, data.frame(a = "side effect name", b = 0))

################################### EXPERIMENT OUTPUT FILE NAME ################################################################
fileName <- paste(experimentName, '.csv')
################################################################################################################################

experimentFile <- paste("C:\\Users\\radeshpa\\Desktop\\BioInformatics\\Assignment3\\Experiment_output", fileName)

for(i in 1:length(sideEffectNames)){
    sideEffectName <- sideEffectNames[i]
    cat("Working on the side effect: ", sideEffectName, "\n")
    currentSideEffect <- sideEffectData[,sideEffectName]
    presentIndices <- which(currentSideEffect == 1)
    absentIndices <- which(currentSideEffect == 0)

   
    outerCount <- 3
    accuracyOverFoldsTotal <- 0
    
    for(j in 1:outerCount){
        
################################################  DATA  #######################################################################
        
        foldCount <- 10
        folds <- createFolds(currentSideEffect, foldCount, returnTrain = FALSE)
        # Get the values for every list (indices)

        foldIndices <- list()
        
        for(k in 1:length(folds)){
            foldIndices[[length(foldIndices)+1]] <- folds[[k]]
        }

        accuracyOverFolds <- 0
        accuracyOverFoldsSum <- 0
       
        
        for(k in 1:length(folds)){

            # Data of this fold.
#            cat("Running fold number: ", k, " Count number: ", j, "\n")

            # We need to create a vector for indexing
            testingDataX <- myData[foldIndices[[k]],]

            testingDataY <- sideEffectData[foldIndices[[k]], sideEffectName]

            remainingIndices <- (Reduce(c,foldIndices[-k]))

            trainingDataX <- myData[c(remainingIndices), ]

            trainingDataY <- sideEffectData[c(remainingIndices), sideEffectName]

##################################################### FEATURE SELECTION : CORRELATION ###########################################################            

            correlationResults <- apply(trainingDataX, 2, function(x){ cor(x, trainingDataY)})
            boundary <- 0.05
            geneIndices  <- c(which(correlationResults  > boundary), which(correlationResults < -boundary))

            featuresOfInterest <- names(trainingDataX)[geneIndices]

            refinedTrainingDataX <- trainingDataX[, featuresOfInterest]

            
########################################## SVM #######################################################################################

#            cat("Features of interest: ", length(featuresOfInterest), "\n")
            # Now filter the test set also!
            refinedTestingDataX <- testingDataX[,featuresOfInterest]
            svmModel <- svm(refinedTrainingDataX, trainingDataY)
            predictionResults <- predict(svmModel, refinedTestingDataX)
#            cat("Count of prediction results: ", length(predictionResults))

            probabilityThreshold <- 0.5
            predictionResults <- round(predictionResults) # default is 0.5
            accuracy <- (sum(predictionResults==testingDataY) + sum(is.na(predictionResults) & is.na(testingDataY))) / length(predictionResults)
            #cat("Accuracy for fold:  ", i, " is ", accuracy, "\n")
            accuracyOverFoldsSum <- accuracyOverFoldsSum + accuracy
            
#######################################################################################################################
           
        }
        
        accuracyOverFolds <- (accuracyOverFoldsSum)/length(folds)     
        accuracyOverFoldsTotal <- accuracyOverFoldsTotal + accuracyOverFolds
    }
    
    finalAccuracy  <- accuracyOverFoldsTotal/outerCount
#    cat("Final Accuracy is: ", finalAccuracy, "\n")
    
    write.table(data.frame(a = sideEffectName, b = finalAccuracy), experimentFile, append = TRUE, sep = "\t", row.names = FALSE, col.names = FALSE)    
}

end.time <- Sys.time()
cat("took time: ",end.time - start.time)

### Experiment 5.3,  Correlation of 0.05, -0.05, knn = 10

In [1]:
start.time <- Sys.time()
sideEffectNames <- names(sideEffectData)
geneNames <- names(myData)
experimentName <- "correlation_0_05_knn_10_Accuracy"
experimentResults <- data.frame(a = character(), b= numeric())
experimentResults <- rbind(experimentResults, data.frame(a = "side effect name", b = 0))

################################### EXPERIMENT OUTPUT FILE NAME ################################################################
fileName <- paste(experimentName, '.csv')
################################################################################################################################

experimentFile <- paste("C:\\Users\\radeshpa\\Desktop\\BioInformatics\\Assignment3\\Experiment_output", fileName)

for(i in 1:length(sideEffectNames)){
    sideEffectName <- sideEffectNames[i]
    cat("Working on the side effect: ", sideEffectName, "\n")
    currentSideEffect <- sideEffectData[,sideEffectName]
    presentIndices <- which(currentSideEffect == 1)
    absentIndices <- which(currentSideEffect == 0)

   
    outerCount <- 3
    accuracyOverFoldsTotal <- 0
    
    for(j in 1:outerCount){
        
################################################  DATA  #######################################################################
        
        foldCount <- 10
        folds <- createFolds(currentSideEffect, foldCount, returnTrain = FALSE)
        # Get the values for every list (indices)

        foldIndices <- list()
        
        for(k in 1:length(folds)){
            foldIndices[[length(foldIndices)+1]] <- folds[[k]]
        }

        accuracyOverFolds <- 0
        accuracyOverFoldsSum <- 0
       
        
        for(k in 1:length(folds)){

            # Data of this fold.
#            cat("Running fold number: ", k, " Count number: ", j, "\n")

            # We need to create a vector for indexing
            testingDataX <- myData[foldIndices[[k]],]

            testingDataY <- sideEffectData[foldIndices[[k]], sideEffectName]

            remainingIndices <- (Reduce(c,foldIndices[-k]))

            trainingDataX <- myData[c(remainingIndices), ]

            trainingDataY <- sideEffectData[c(remainingIndices), sideEffectName]

##################################################### FEATURE SELECTION : CORRELATION ###########################################################            

            correlationResults <- apply(trainingDataX, 2, function(x){ cor(x, trainingDataY)})
            boundary <- 0.05
            geneIndices  <- c(which(correlationResults  > boundary), which(correlationResults < -boundary))

             featuresOfInterest <- names(trainingDataX)[geneIndices]

            refinedTrainingDataX <- trainingDataX[, featuresOfInterest]

            
########################################## KNN, K = 10 #######################################################################################

            # Now filter the test set also!
            refinedTestingDataX <- testingDataX[,featuresOfInterest]
            predictionResults <- knn(refinedTrainingDataX, refinedTestingDataX, factor(trainingDataY), k = 10)

            accuracy <- (sum(predictionResults==testingDataY) + sum(is.na(predictionResults) & is.na(testingDataY))) / length(predictionResults)
             accuracyOverFoldsSum <- accuracyOverFoldsSum + accuracy

     
#######################################################################################################################        
           
        }
        
        accuracyOverFolds <- (accuracyOverFoldsSum)/length(folds)     
        accuracyOverFoldsTotal <- accuracyOverFoldsTotal + accuracyOverFolds
    }
    
    finalAccuracy  <- accuracyOverFoldsTotal/outerCount
#    cat("Final Accuracy is: ", finalAccuracy, "\n")
    
    write.table(data.frame(a = sideEffectName, b = finalAccuracy), experimentFile, append = TRUE, sep = "\t", row.names = FALSE, col.names = FALSE)    
}

end.time <- Sys.time()
cat("took time: ",end.time - start.time)

ERROR: Error in eval(expr, envir, enclos): object 'sideEffectData' not found


### Experiment 5.4, Correlation 0.10, -0.10, SVM

In [3]:
start.time <- Sys.time()
sideEffectNames <- names(sideEffectData)
geneNames <- names(myData)
experimentName <- "correlation_0_10_SVM_Accuracy"
experimentResults <- data.frame(a = character(), b= numeric())
experimentResults <- rbind(experimentResults, data.frame(a = "side effect name", b = 0))

################################### EXPERIMENT OUTPUT FILE NAME ################################################################
fileName <- paste(experimentName, '.csv')
################################################################################################################################

experimentFile <- paste("C:\\Users\\radeshpa\\Desktop\\BioInformatics\\Assignment3\\Experiment_output", fileName)

for(i in 1:length(sideEffectNames)){
    sideEffectName <- sideEffectNames[i]
    cat("Working on the side effect: ", sideEffectName, "\n")
    currentSideEffect <- sideEffectData[,sideEffectName]
    presentIndices <- which(currentSideEffect == 1)
    absentIndices <- which(currentSideEffect == 0)

   
    outerCount <- 3
    accuracyOverFoldsTotal <- 0
    
    for(j in 1:outerCount){
        
################################################  DATA  #######################################################################
        
        foldCount <- 10
        folds <- createFolds(currentSideEffect, foldCount, returnTrain = FALSE)
        # Get the values for every list (indices)

        foldIndices <- list()
        
        for(k in 1:length(folds)){
            foldIndices[[length(foldIndices)+1]] <- folds[[k]]
        }

        accuracyOverFolds <- 0
        accuracyOverFoldsSum <- 0
       
        
        for(k in 1:length(folds)){

            # Data of this fold.
#            cat("Running fold number: ", k, " Count number: ", j, "\n")

            # We need to create a vector for indexing
            testingDataX <- myData[foldIndices[[k]],]

            testingDataY <- sideEffectData[foldIndices[[k]], sideEffectName]

            remainingIndices <- (Reduce(c,foldIndices[-k]))

            trainingDataX <- myData[c(remainingIndices), ]

            trainingDataY <- sideEffectData[c(remainingIndices), sideEffectName]

##################################################### FEATURE SELECTION : CORRELATION ###########################################################            

            correlationResults <- apply(trainingDataX, 2, function(x){ cor(x, trainingDataY)})
            boundary <- 0.10
            geneIndices  <- c(which(correlationResults  > boundary), which(correlationResults < -boundary))

            featuresOfInterest <- names(trainingDataX)[geneIndices]

            refinedTrainingDataX <- trainingDataX[, featuresOfInterest]

            
########################################## SVM #######################################################################################

#            cat("Features of interest: ", length(featuresOfInterest), "\n")
            # Now filter the test set also!
            refinedTestingDataX <- testingDataX[,featuresOfInterest]
            svmModel <- svm(refinedTrainingDataX, trainingDataY)
            predictionResults <- predict(svmModel, refinedTestingDataX)
#            cat("Count of prediction results: ", length(predictionResults))

            probabilityThreshold <- 0.5
            predictionResults <- round(predictionResults) # default is 0.5
            accuracy <- (sum(predictionResults==testingDataY) + sum(is.na(predictionResults) & is.na(testingDataY))) / length(predictionResults)
            #cat("Accuracy for fold:  ", i, " is ", accuracy, "\n")
            accuracyOverFoldsSum <- accuracyOverFoldsSum + accuracy
            
#######################################################################################################################
           
        }
        
        accuracyOverFolds <- (accuracyOverFoldsSum)/length(folds)     
        accuracyOverFoldsTotal <- accuracyOverFoldsTotal + accuracyOverFolds
    }
    
    finalAccuracy  <- accuracyOverFoldsTotal/outerCount
#    cat("Final Accuracy is: ", finalAccuracy, "\n")
    
    write.table(data.frame(a = sideEffectName, b = finalAccuracy), experimentFile, append = TRUE, sep = "\t", row.names = FALSE, col.names = FALSE)    
}

end.time <- Sys.time()
cat("took time: ",end.time - start.time)

ERROR: Error in eval(expr, envir, enclos): object 'sideEffectData' not found


### Experiment 5.5,  t-test with p-value < 0.03, knn = 10

In [ ]:
start.time <- Sys.time()
sideEffectNames <- names(sideEffectData)
geneNames <- names(myData)
experimentName <- "t_0_03_kNN_k_10"
experimentResults <- data.frame(a = character(), b= numeric())
experimentResults <- rbind(experimentResults, data.frame(a = "side effect name", b = 0))

################################### EXPERIMENT OUTPUT FILE NAME ################################################################
fileName <- paste(experimentName, '.csv')
################################################################################################################################

experimentFile <- paste("C:\\Users\\radeshpa\\Desktop\\BioInformatics\\Assignment3\\Experiment_output", fileName)

for(i in 215:length(sideEffectNames)){
    sideEffectName <- sideEffectNames[i]
    cat("Working on the side effect: ", sideEffectName, "\n")
    currentSideEffect <- sideEffectData[,sideEffectName]
    presentIndices <- which(currentSideEffect == 1)
    absentIndices <- which(currentSideEffect == 0)
   
    outerCount <- 3
    accuracyOverFoldsTotal <- 0
    
    for(j in 1:outerCount){
        
        
################################################  DATA  #######################################################################
        
        foldCount <- 10
        folds <- createFolds(currentSideEffect, foldCount, returnTrain = FALSE)
        # Get the values for every list (indices)

        foldIndices <- list()
        
        for(k in 1:length(folds)){
            foldIndices[[length(foldIndices)+1]] <- folds[[k]]
        }

        accuracyOverFolds <- 0
        accuracyOverFoldsSum <- 0
       
        
        for(k in 1:length(folds)){
            # Data of this fold.
            cat("Running fold number: ", k, " Count number: ", j, "\n")

            # We need to create a vector for indexing
            testingDataX <- myData[foldIndices[[k]],]


            testingDataY <- sideEffectData[foldIndices[[k]],sideEffectName]


            remainingIndices <- (Reduce(c,foldIndices[-k]))

            trainingDataX <- myData[c(remainingIndices), ]

            trainingDataY <- sideEffectData[c(remainingIndices), sideEffectName]

##################################################### FEATURE SELECTION : t- statistic ###########################################################            

            allpValues <- sapply(geneNames, function(geneName){
                geneData <- trainingDataX[,geneName]
                sampleOne <-  geneData[presentIndices]
                sampleOneLength <- length(sampleOne)
                sampleTwo <-  geneData[absentIndices]
                sampleTwoLength <- length(sampleTwo)
                if(sampleOneLength == 0 | sampleTwoLength == 0 | (sampleOneLength == 1 && sampleTwoLength != 1) | (sampleOneLength != 1 && sampleTwoLength == 1)){
                    pValue  <- 0 # we want this feature.
                }
                else{
                    pValue <- t.test(sampleOne, sampleTwo)$p.value
                }
                pValue            
            })

            ######## THRESHOLD FOR THE STATISTIC #############
            
            threshold <- 0.03
            sigpIndices <- which(allpValues < threshold)

            sigpValues <- allpValues[sigpIndices]
            featuresOfInterest <- names(trainingDataX)[sigpIndices]

            refinedTrainingDataX <- trainingDataX[, featuresOfInterest]

            
########################################## KNN, K = 10 #######################################################################################

            # Now filter the test set also!
            refinedTestingDataX <- testingDataX[,featuresOfInterest]
            predictionResults <- knn(refinedTrainingDataX, refinedTestingDataX, factor(trainingDataY), k = 10)

            accuracy <- (sum(predictionResults==testingDataY) + sum(is.na(predictionResults) & is.na(testingDataY))) / length(predictionResults)
            accuracyOverFoldsSum <- accuracyOverFoldsSum + accuracy
     
#######################################################################################################################        

        }
        
        accuracyOverFolds <- (accuracyOverFoldsSum)/length(folds)     
        accuracyOverFoldsTotal <- accuracyOverFoldsTotal + accuracyOverFolds
    }
    
    finalAccuracy  <- accuracyOverFoldsTotal/outerCount
    cat("Final Accuracy is: ", finalAccuracy, "\n")
    
    write.table(data.frame(a = sideEffectName, b = finalAccuracy), experimentFile, append = TRUE, sep = "\t", row.names = FALSE, col.names = FALSE)    
}

end.time <- Sys.time()
cat("took time: ",end.time - start.time)

### Experiment 5.6, t-test with p-value < 0.03, SVM

In [ ]:
start.time <- Sys.time()
sideEffectNames <- names(sideEffectData)
geneNames <- names(myData)
experimentName <- "t_0_03_svm"
experimentResults <- data.frame(a = character(), b= numeric())
experimentResults <- rbind(experimentResults, data.frame(a = "side effect name", b = 0))

################################### EXPERIMENT OUTPUT FILE NAME ################################################################
fileName <- paste(experimentName, '.csv')
################################################################################################################################

experimentFile <- paste("C:\\Users\\radeshpa\\Desktop\\BioInformatics\\Assignment3\\Experiment_output", fileName)

for(i in 1:length(sideEffectNames)){
    sideEffectName <- sideEffectNames[i]
    cat("Working on the side effect: ", sideEffectName, "\n")
    currentSideEffect <- sideEffectData[,sideEffectName]
    presentIndices <- which(currentSideEffect == 1)
    absentIndices <- which(currentSideEffect == 0)
   
    outerCount <- 3
    accuracyOverFoldsTotal <- 0
    
    for(j in 1:outerCount){
        
        
################################################  DATA  #######################################################################
        
        foldCount <- 10
        folds <- createFolds(currentSideEffect, foldCount, returnTrain = FALSE)
        # Get the values for every list (indices)

        foldIndices <- list()
        
        for(k in 1:length(folds)){
            foldIndices[[length(foldIndices)+1]] <- folds[[k]]
        }

        accuracyOverFolds <- 0
        accuracyOverFoldsSum <- 0
       
        
        for(k in 1:length(folds)){
            # Data of this fold.
            cat("Running fold number: ", k, " Count number: ", j, "\n")

            # We need to create a vector for indexing
            testingDataX <- myData[foldIndices[[k]],]


            testingDataY <- sideEffectData[foldIndices[[k]],sideEffectName]


            remainingIndices <- (Reduce(c,foldIndices[-k]))

            trainingDataX <- myData[c(remainingIndices), ]

            trainingDataY <- sideEffectData[c(remainingIndices), sideEffectName]

##################################################### FEATURE SELECTION : t- statistic ###########################################################            

            allpValues <- sapply(geneNames, function(geneName){
                geneData <- trainingDataX[,geneName]
                sampleOne <-  geneData[presentIndices]
                sampleOneLength <- length(sampleOne)
                sampleTwo <-  geneData[absentIndices]
                sampleTwoLength <- length(sampleTwo)
                if(sampleOneLength == 0 | sampleTwoLength == 0 | (sampleOneLength == 1 && sampleTwoLength != 1) | (sampleOneLength != 1 && sampleTwoLength == 1)){
                    pValue  <- 0 # we want this feature.
                }
                else{
                    pValue <- t.test(sampleOne, sampleTwo)$p.value
                }
                pValue            
            })

            ######## THRESHOLD FOR THE STATISTIC #############
            
            threshold <- 0.03
            sigpIndices <- which(allpValues < threshold)

            sigpValues <- allpValues[sigpIndices]
            featuresOfInterest <- names(trainingDataX)[sigpIndices]

            refinedTrainingDataX <- trainingDataX[, featuresOfInterest]

            
########################################## SVM #######################################################################################

#            cat("Features of interest: ", length(featuresOfInterest), "\n")
            # Now filter the test set also!
            refinedTestingDataX <- testingDataX[,featuresOfInterest]
            svmModel <- svm(refinedTrainingDataX, trainingDataY)
            predictionResults <- predict(svmModel, refinedTestingDataX)
#            cat("Count of prediction results: ", length(predictionResults))

            probabilityThreshold <- 0.5
            predictionResults <- round(predictionResults) # default is 0.5
            accuracy <- (sum(predictionResults==testingDataY) + sum(is.na(predictionResults) & is.na(testingDataY))) / length(predictionResults)
            #cat("Accuracy for fold:  ", i, " is ", accuracy, "\n")
            accuracyOverFoldsSum <- accuracyOverFoldsSum + accuracy
     
#######################################################################################################################        

        }
        
        accuracyOverFolds <- (accuracyOverFoldsSum)/length(folds)     
        accuracyOverFoldsTotal <- accuracyOverFoldsTotal + accuracyOverFolds
    }
    
    finalAccuracy  <- accuracyOverFoldsTotal/outerCount
    cat("Final Accuracy is: ", finalAccuracy, "\n")
    
    write.table(data.frame(a = sideEffectName, b = finalAccuracy), experimentFile, append = TRUE, sep = "\t", row.names = FALSE, col.names = FALSE)    
}

end.time <- Sys.time()
cat("took time: ",end.time - start.time)

### Experiment 5.7, No Feature Selection, SVM

In [ ]:
start.time <- Sys.time()
sideEffectNames <- names(sideEffectData)
geneNames <- names(myData)
experimentName <- "none_SVM"
experimentResults <- data.frame(a = character(), b= numeric())
experimentResults <- rbind(experimentResults, data.frame(a = "side effect name", b = 0))

################################### EXPERIMENT OUTPUT FILE NAME ################################################################

fileName <- paste(experimentName, '.csv')

################################################################################################################################

experimentFile <- paste("C:\\Users\\radeshpa\\Desktop\\BioInformatics\\Assignment3\\Experiment_output", fileName)

# For every side effect (the output/outcome),  perform 10 fold cross validation. Also perform feature selection and testing at 
# every fold.

cat("Count of side effects is: ", length(sideEffectNames), "\n")

for(i in 1:length(sideEffectNames)){
    sideEffectName <- sideEffectNames[i]
    cat("Working on the side effect: ", sideEffectName, "\n")
    currentSideEffect <- sideEffectData[,sideEffectName]
    presentIndices <- which(currentSideEffect == 1)
    absentIndices <- which(currentSideEffect == 0)
   
    outerCount <- 3
    accuracyOverFoldsTotal <- 0
    
    
    for(j in 1:outerCount){
        
        
################################################  DATA  #######################################################################
        
        foldCount <- 10
        folds <- createFolds(currentSideEffect, foldCount, returnTrain = FALSE)
        # Get the values for every list (indices)

        foldIndices <- list()
        
        for(k in 1:length(folds)){
            foldIndices[[length(foldIndices)+1]] <- folds[[k]]
        }

        accuracyOverFolds <- 0
        accuracyOverFoldsSum <- 0
       
        
        for(k in 1:length(folds)){

            # Data of this fold.
#            cat("Running fold number: ", k, " Count number: ", j, "\n")
            
            # Create the frames for the training data and the testing data.
            
            # We need to create a vector for indexing
            testingDataX <- myData[foldIndices[[k]],]


            testingDataY <- sideEffectData[foldIndices[[k]], sideEffectName]


            remainingIndices <- (Reduce(c,foldIndices[-k]))

            trainingDataX <- myData[c(remainingIndices), ]

            trainingDataY <- sideEffectData[c(remainingIndices), sideEffectName]


            featuresOfInterest <- names(trainingDataX)

            refinedTrainingDataX <- trainingDataX[, featuresOfInterest]

            
########################################## SVM #######################################################################################

#            cat("Features of interest: ", length(featuresOfInterest), "\n")
            # Now filter the test set also!
            refinedTestingDataX <- testingDataX[,featuresOfInterest]
            svmModel <- svm(refinedTrainingDataX, trainingDataY)
            predictionResults <- predict(svmModel, refinedTestingDataX)
#            cat("Count of prediction results: ", length(predictionResults))

            probabilityThreshold <- 0.5
            predictionResults <- round(predictionResults) # default is 0.5
            accuracy <- (sum(predictionResults==testingDataY) + sum(is.na(predictionResults) & is.na(testingDataY))) / length(predictionResults)
            #cat("Accuracy for fold:  ", i, " is ", accuracy, "\n")
            accuracyOverFoldsSum <- accuracyOverFoldsSum + accuracy
            
#######################################################################################################################
           
        }
        
        accuracyOverFolds <- (accuracyOverFoldsSum)/length(folds)     
        accuracyOverFoldsTotal <- accuracyOverFoldsTotal + accuracyOverFolds
    }
    
    finalAccuracy  <- accuracyOverFoldsTotal/outerCount
#    cat("Final Accuracy is: ", finalAccuracy, "\n")
    
    write.table(data.frame(a = sideEffectName, b = finalAccuracy), experimentFile, append = TRUE, sep = "\t", row.names = FALSE, col.names = FALSE)    
}

end.time <- Sys.time()
cat("took time: ",end.time - start.time)

### Experiment t-test with p-value < 0.05, SVM

In [ ]:
start.time <- Sys.time()
sideEffectNames <- names(sideEffectData)
geneNames <- names(myData)
experimentName <- "t_0_05_SVM"
experimentResults <- data.frame(a = character(), b= numeric())
experimentResults <- rbind(experimentResults, data.frame(a = "side effect name", b = 0))

################################### EXPERIMENT OUTPUT FILE NAME ################################################################

fileName <- paste(experimentName, '.csv')

################################################################################################################################

experimentFile <- paste("C:\\Users\\radeshpa\\Desktop\\BioInformatics\\Assignment3\\Experiment_output", fileName)

# For every side effect (the output/outcome),  perform 10 fold cross validation. Also perform feature selection and testing at 
# every fold.

cat("Count of side effects is: ", length(sideEffectNames), "\n")

for(i in 1:length(sideEffectNames)){
    sideEffectName <- sideEffectNames[i]
    cat("Working on the side effect: ", sideEffectName, "\n")
    currentSideEffect <- sideEffectData[,sideEffectName]
    presentIndices <- which(currentSideEffect == 1)
    absentIndices <- which(currentSideEffect == 0)
   
    outerCount <- 3
    accuracyOverFoldsTotal <- 0
    
    
    for(j in 1:outerCount){
        
        
################################################  DATA  #######################################################################
        
        foldCount <- 10
        folds <- createFolds(currentSideEffect, foldCount, returnTrain = FALSE)
        # Get the values for every list (indices)

        foldIndices <- list()
        
        for(k in 1:length(folds)){
            foldIndices[[length(foldIndices)+1]] <- folds[[k]]
        }

        accuracyOverFolds <- 0
        accuracyOverFoldsSum <- 0
       
        
        for(k in 1:length(folds)){

            # Data of this fold.
#            cat("Running fold number: ", k, " Count number: ", j, "\n")
            
            # Create the frames for the training data and the testing data.
            
            # We need to create a vector for indexing
            testingDataX <- myData[foldIndices[[k]],]


            testingDataY <- sideEffectData[foldIndices[[k]], sideEffectName]


            remainingIndices <- (Reduce(c,foldIndices[-k]))

            trainingDataX <- myData[c(remainingIndices), ]

            trainingDataY <- sideEffectData[c(remainingIndices), sideEffectName]

##################################################### FEATURE SELECTION : t- statistic ###########################################################            

            allpValues <- sapply(geneNames, function(geneName){
                geneData <- trainingDataX[,geneName]
                sampleOne <-  geneData[presentIndices]
                sampleOneLength <- length(sampleOne)
                sampleTwo <-  geneData[absentIndices]
                sampleTwoLength <- length(sampleTwo)

                if(sampleOneLength == 0 | sampleTwoLength == 0 | (sampleOneLength == 1 && sampleTwoLength != 1) | (sampleOneLength != 1 && sampleTwoLength == 1)){
                    # this is a great feature!
                    pValue  <- 0 # we want this feature.
                }
                else{

                    pValue <- t.test(sampleOne, sampleTwo)$p.value

                } 
                pValue            
            })


            threshold <- 0.05
            sigpIndices <- which(allpValues < threshold)

            sigpValues <- allpValues[sigpIndices]
            featuresOfInterest <- names(trainingDataX)[sigpIndices]

            refinedTrainingDataX <- trainingDataX[, featuresOfInterest]

            
########################################## SVM #######################################################################################

#            cat("Features of interest: ", length(featuresOfInterest), "\n")
            # Now filter the test set also!
            refinedTestingDataX <- testingDataX[,featuresOfInterest]
            svmModel <- svm(refinedTrainingDataX, trainingDataY)
            predictionResults <- predict(svmModel, refinedTestingDataX)
#            cat("Count of prediction results: ", length(predictionResults))

            probabilityThreshold <- 0.5
            predictionResults <- round(predictionResults) # default is 0.5
            accuracy <- (sum(predictionResults==testingDataY) + sum(is.na(predictionResults) & is.na(testingDataY))) / length(predictionResults)
            #cat("Accuracy for fold:  ", i, " is ", accuracy, "\n")
            accuracyOverFoldsSum <- accuracyOverFoldsSum + accuracy
            
#######################################################################################################################
           
        }
        
        accuracyOverFolds <- (accuracyOverFoldsSum)/length(folds)     
        accuracyOverFoldsTotal <- accuracyOverFoldsTotal + accuracyOverFolds
    }
    
    finalAccuracy  <- accuracyOverFoldsTotal/outerCount
#    cat("Final Accuracy is: ", finalAccuracy, "\n")
    
    write.table(data.frame(a = sideEffectName, b = finalAccuracy), experimentFile, append = TRUE, sep = "\t", row.names = FALSE, col.names = FALSE)    
}

end.time <- Sys.time()
cat("took time: ",end.time - start.time)

## Questions and Answers

#### Q1) Here are the side effects we can predict with the highest accuracy with each approach:
1.  t = 0.05, knn, k = 10: Gastrointestinal.tract.disorders.congenital
2. t = 0.03, knn, k = 10: Gastrointestinal.tract.disorders.congenital
3. t = 0.05, SVM: Gastrointestinal.tract.disorders.congenital
4. t = 0.03, SVM: Gastrointestinal.tract.disorders.congenital
5. none, SVM: Gastrointestinal.tract.disorders.congenital
6. correlation = 0.10, SVM: Gastrointestinal.tract.disorders.congenital
7. correlation = 0.05, SVM: Gastrointestinal.tract.disorders.congenital
8. correlation = 0.05, knn, k = 10 : Gastrointestinal.tract.disorders.congenital

#### Q2) Here are the side effects we can predict with the lowest accuracy with each approach:
1.  t = 0.05, knn, k = 10: Arteriosclerosis..stenosis..vascular.insufficiency.and.necrosis
2. t = 0.03, knn, k = 10: Heart.failures
3. t = 0.05, SVM: Heart.failures
4. t = 0.03, SVM: Deliria..incl.confusion.
5. none, SVM: Heart.failures
6. correlation = 0.10, SVM: Movement.disorders..incl.parkinsonism.
7. correlation = 0.05, SVM: Seizures..incl.subtypes.
8. correlation = 0.05, knn, k = 10 : Movement.disorders..incl.parkinsonism.

#### Q3) Yes, there are such cases. Ideally we would use the F-1 score rather than the accuracy to calculate the performance of our model.

#### Q4) Ideally the best model would be one that offers the best performance for every side effect. Also when we are comparing multiple models,
we can use the RMSE metric. In this case, since I already know the accuracy of all the side effects for every model, I simply chose the model
with the highest overall average accuracy. I think this is a good start.

And the answer is the model is: t = 0.05, SVM

#### Q5) Yes. I noticed that when I did not perform feature selection, then it took forever for the training and testing phases to finish. This makes sense because now  the model is soo big, since there are soo many features  - a majority of which may not even be relevant. Also if I did some further data analysis using Power BI for the data  for the side effects for which the different models did terribly (or had low accuracy), I think I can see some data patterns and understand as to why the accuracies are low for these side effects